In [ ]:
!pip install llama_index transformers unstructured pymilvus
!pip install llama-index-core
!pip install llama-index-extractors-entity
!pip install llama-index-vector-stores-milvus
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-dashscope
!pip install llama-index-extractors
!pip install pymilvus[milvus_lite]
!pip install unstructured[docx]
!pip install unstructured[doc]
!pip install unstructured[txt]
!pip install unstructured[md]
!pip install fitz frontend tools
!pip uninstall fitz pymupdf -y
!pip install pymupdf
!pip install -r requirements.txt

In [1]:
from llama_index.core import (VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage
    , Document, Settings, StorageContext, PromptTemplate)
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.extractors import KeywordExtractor, SummaryExtractor
from llama_index.core.schema import MetadataMode
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.dashscope import DashScope
from llama_index.llms.openai import OpenAI

from llama_index.extractors.entity import EntityExtractor
from llama_index.readers.file import UnstructuredReader,PyMuPDFReader,PDFReader

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

import os, re, asyncio
from tqdm.asyncio import tqdm_asyncio
from tqdm import tqdm
import json

/root/miniconda3/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
#!python pdf2md.py

In [3]:
embedding_model = "./Qwen3-Embedding-0.6B"
Settings.embed_model = HuggingFaceEmbedding(
    model_name=embedding_model,
    cache_folder=None,
    trust_remote_code=True,
    local_files_only=True
)

config = AutoConfig.from_pretrained(embedding_model, trust_remote_code=True, local_files_only=True)
dimension = config.hidden_size
print(f"模型嵌入维度: {dimension}")

2025-10-11 09:49:31,908 - INFO - Load pretrained SentenceTransformer: ./Qwen3-Embedding-0.6B
2025-10-11 09:49:32,886 - INFO - 1 prompt is loaded, with the key: query


模型嵌入维度: 1024


In [18]:
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    LLMMetadata,
)
from llama_index.core.llms.callbacks import llm_completion_callback
from llama_index.core import Settings
from typing import Any
import requests


class SiliconFlowLLM(CustomLLM):
    """硅基流动自定义 LLM"""
    
    model: str = "Qwen/Qwen3-Next-80B-A3B-Instruct"
    api_key: str = ""
    api_base: str = "https://api.siliconflow.cn/v1"
    max_tokens: int = 4096
    temperature: float = 0.1
    
    @property
    def metadata(self) -> LLMMetadata:
        """获取 LLM 元数据"""
        return LLMMetadata(
            context_window=32768,  # 根据具体模型调整
            num_output=self.max_tokens,
            model_name=self.model,
        )
    
    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        """完成请求"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "max_tokens": kwargs.get("max_tokens", self.max_tokens),
            "temperature": kwargs.get("temperature", self.temperature),
            "stream": False
        }
        
        response = requests.post(
            f"{self.api_base}/chat/completions",
            headers=headers,
            json=data
        )
        
        response.raise_for_status()
        result = response.json()
        
        return CompletionResponse(
            text=result["choices"][0]["message"]["content"]
        )
    
    @llm_completion_callback()
    def stream_complete(self, prompt: str, **kwargs: Any):
        """流式完成（未实现，但需要定义）"""
        # 调用非流式方法
        response = self.complete(prompt, **kwargs)
        yield response


# 使用示例
if __name__ == "__main__":
    # 1. 创建自定义 LLM 实例
    llm = SiliconFlowLLM(
        model="Qwen/Qwen3-30B-A3B-Instruct-2507",  # 可选其他模型
        api_key="sk-ionsbeieleeekwlstqotkyrmictdzshgnbaytavcudxkixcs",  # 替换为你的 API Ke
        api_base = "https://api.siliconflow.cn/v1",
        max_tokens=1024,
        temperature=0.3
    )
    
    # 2. 设置到 Settings
    Settings.llm = llm
    
    # 3. 测试使用
    response = llm.complete("你好，请介绍一下你自己")
    print(response.text)


你好！我是通义千问（Qwen），是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，欢迎随时告诉我！


In [5]:
milvus_dir = "./milvus_test"
milvus_db_path = os.path.join(milvus_dir, "milvus_lite.db")
abs_db_path = os.path.abspath(milvus_db_path)
print(f"绝对数据库路径: {abs_db_path}")

if not os.path.exists(milvus_dir):
    os.makedirs(milvus_dir)
    print("已创建 ./milvus 目录")



# milvus_vector_store = MilvusVectorStore(
#     uri=f"{abs_db_path}",
#     collection_name="rag_collection",
#     dim=1024,
#     overwrite=True
# )
# storage_context = StorageContext.from_defaults(vector_store=milvus_vector_store)

绝对数据库路径: /root/milvus_test/milvus_lite.db


### 首次运行

In [5]:
###首次运行
milvus_dir = "./milvus_test"
milvus_db_path = os.path.join(milvus_dir, "milvus_lite.db")
abs_db_path = os.path.abspath(milvus_db_path)
print(f"绝对数据库路径: {abs_db_path}")

if not os.path.exists(milvus_dir):
    os.makedirs(milvus_dir)
    print("已创建 ./milvus 目录")



milvus_vector_store = MilvusVectorStore(
    uri=f"{abs_db_path}",
    collection_name="rag_collection",
    dim=1024,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=milvus_vector_store)

绝对数据库路径: /root/milvus_test/milvus_lite.db


/root/miniconda3/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [6]:
def clean_text(text: str) -> str:
    text = re.sub(r'\n\s*\n+', '\n\n', text).strip()
    # text = re.sub(r'(\w+\s*){3,}\n', '', text)
    # text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fa5\s\.,!?]', '', text)  # 去除特殊字符，保留中英文
    return text


In [7]:
async def generate_summary_async(text, max_words=30):
    prompt = f"总结以下文本，不超过{max_words}字，直接回复结果：{text}"
    response = await Settings.llm.acomplete(prompt)
    return response.text.strip()

def generate_summary(text, max_words=30):
    prompt = f"总结以下文本，不超过{max_words}字，直接回复结果：{text}"
    response = Settings.llm.complete(prompt)
    return response.text.strip()

async def add_summaries_to_nodes_async(nodes_list):
    tasks = [generate_summary_async(node.text) for node in nodes_list]

    summaries = []
    for future in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="生成节点摘要进度"):
        summary = await future
        summaries.append(summary)

    for node, summary in zip(nodes_list, summaries):
        node.metadata["node_summary"] = summary
        
def add_summaries_to_nodes(nodes_list):
    for node in tqdm(nodes_list, desc="生成摘要"):
        summary = generate_summary(node.text)
        node.metadata["node_summary"] = summary

In [8]:
qwen_tokenizer = AutoTokenizer.from_pretrained("./Qwen3-Embedding-0.6B", trust_remote_code=True)
documents_dir = "./docs"

file_extractor = {
    ".docx": UnstructuredReader(),
    ".doc": UnstructuredReader(),
    ".txt": UnstructuredReader(),
    ".md": UnstructuredReader(),
}


In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

def load_single_file(file_path, file_extractor):
    """加载单个文件"""
    try:
        ext = Path(file_path).suffix.lower()
        if ext in file_extractor:
            reader = file_extractor[ext]
            print('loading:',file_path)
            docs = reader.load_data(file_path)
            return docs
        return []
    except Exception as e:
        print(f"加载文件 {file_path} 失败: {e}")
        return []

def load_documents_parallel(documents_dir, file_extractor, max_workers=4):
    """并行加载文档"""
    all_files = []
    for ext in file_extractor.keys():
        all_files.extend(Path(documents_dir).rglob(f"*{ext}"))
    
    documents = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(load_single_file, str(f), file_extractor): f 
                   for f in all_files}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="加载文件"):
            docs = future.result()
            documents.extend(docs)
    
    return documents

In [11]:
def preprocess_long_documents(documents, max_length=100000, overlap=0):
    """预处理超长文档，避免 tokenizer 处理超长文本"""
    processed_docs = []
    for doc in documents:
        text_length = len(doc.text)
        # 如果文档太长，先粗切分
        if text_length > max_length:
            print(f"检测到超长文档: {text_length} 字符，进行预切分")
            # 按固定长度切分，带重叠
            chunks = []
            start = 0
            chunk_index = 0
            
            while start < text_length:
                end = min(start + max_length, text_length)
                chunk_text = doc.text[start:end]
                
                # 创建新的 metadata，添加切片信息
                new_metadata = doc.metadata.copy() if doc.metadata else {}
                new_metadata['chunk_index'] = chunk_index
                new_metadata['total_chunks'] = (text_length + max_length - overlap - 1) // (max_length - overlap)
                new_metadata['is_chunked'] = True
                
                chunks.append(Document(text=chunk_text, metadata=new_metadata))
                
                # 下一个起点：当前起点 + (max_length - overlap)
                # 这样可以保证前后重叠 overlap 个字符
                start += (max_length - overlap)
                chunk_index += 1
            
            processed_docs.extend(chunks)
            print(f"  切分为 {len(chunks)} 个块，每块最大 {max_length} 字符，重叠 {overlap} 字符")
        else:
            processed_docs.append(doc)
    
    return processed_docs

In [15]:
# 使用方法
documents = load_documents_parallel(documents_dir, file_extractor, max_workers=1)

cleaned_documents = [Document(text=clean_text(doc.text), metadata=doc.metadata) 
                     for doc in documents]

# 添加这一步：最大长度100000，前后重叠1000
# cleaned_documents = preprocess_long_documents(
#     cleaned_documents, 
#     max_length=100000, 
#     overlap=0
# )
documents = cleaned_documents

print(f"文件大小:{len(documents)}")

node_parser = SentenceSplitter(chunk_size=1024, chunk_overlap=100, tokenizer=qwen_tokenizer.tokenize)  
nodes = node_parser.get_nodes_from_documents(documents)
print(f"节点数量:{len(nodes)}")

loading:loading: docs/中华人民共和国中国人民银行法.txt
 docs/test_ml.docx
loading: docs/中国人民银行决定实行差别存款准备金率制度.txt
loading: docs/中文新闻 2.txt


加载文件:   6%|▋         | 1/16 [00:00<00:01,  9.82it/s]

loading: docs/中文新闻.txt


2025-10-10 15:11:56,550 - WARNING - 'doc_id' is deprecated and 'id_' will be used instead
加载文件:  12%|█▎        | 2/16 [00:00<00:03,  3.52it/s]

loading: docs/人民币银行结算账户管理办法.txt


...

loading: docs/支付结算办法.md


2025-10-10 15:12:16,851 - WARNING - 'doc_id' is deprecated and 'id_' will be used instead
2025-10-10 15:12:16,863 - WARNING - 'doc_id' is deprecated and 'id_' will be used instead
加载文件:  69%|██████▉   | 11/16 [00:20<00:10,  2.17s/it]2025-10-10 15:12:16,996 - WARNING - 'doc_id' is deprecated and 'id_' will be used instead


loading: docs/腾讯年报大繁体.md
loading: docs/腾讯财报 2025 简体.md


2025-10-10 15:12:18,384 - WARNING - 'doc_id' is deprecated and 'id_' will be used instead
加载文件: 100%|██████████| 16/16 [00:23<00:00,  1.48s/it]


文件大小:16
节点数量:2412


In [ ]:
#add_summaries_to_nodes(nodes)

In [ ]:
# def save_summaries_to_json(nodes_list, file_path="nodes_summaries_temp.json"):
#     summaries_dict = {}
#     for idx, node in enumerate(nodes_list):
#         summaries_dict[str(idx)] = node.metadata.get("node_summary", "")  # 获取摘要，若无则为空
    
#     # 保存到 JSON
#     with open(file_path, 'w', encoding='utf-8') as f:
#         json.dump(summaries_dict, f, ensure_ascii=False, indent=4)
    
#     print(f"节点摘要已保存到 {file_path}")

# def load_summaries_to_nodes(nodes_list, file_path="nodes_summaries.json"):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         summaries_dict = json.load(f)
#     sorted_keys = sorted(summaries_dict.keys(), key=int)

#     for key in sorted_keys:
#         idx = int(key)
#         if idx < len(nodes_list):
#             nodes_list[idx].metadata["node_summary"] = summaries_dict[key]
#         else:
#             print(f"警告：索引 {idx} 超出节点列表长度，跳过。")
    
#     return nodes_list

In [ ]:
# save_summaries_to_json(nodes)

In [13]:
# ============ 保存 Nodes ============
def save_nodes(nodes, save_dir="./saved_nodes"):
    """保存节点数据（支持pickle和json两种格式）"""
    save_path = Path(save_dir)
    save_path.mkdir(parents=True, exist_ok=True)
    
    # 方法1: 使用 pickle 保存完整节点对象（推荐）
    pickle_file = save_path / "nodes.pkl"
    with open(pickle_file, 'wb') as f:
        pickle.dump(nodes, f)
    print(f"Nodes已保存到: {pickle_file}")

def load_nodes(save_dir="./saved_data"):
    """加载节点数据"""
    save_path = Path(save_dir)
    pickle_file = save_path / "nodes.pkl"
    
    if not pickle_file.exists():
        raise FileNotFoundError(f"❌ 找不到节点文件: {pickle_file}")
    
    with open(pickle_file, 'rb') as f:
        nodes = pickle.load(f)
    
    print(f"✅ 已加载 {len(nodes)} 个节点")
    
    # 验证数据
    print(f"📊 节点验证:")
    print(f"  - 总节点数: {len(nodes)}")
    return nodes


In [14]:
save_nodes(nodes, save_dir="./saved_nodes")

Nodes已保存到: saved_nodes/nodes.pkl


In [15]:
# index = VectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     embed_model=Settings.embed_model,
#     node_parser=node_parser,
#     store_nodes_override=True
# )



transformations = [node_parser]
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    embed_model=Settings.embed_model,
    node_parser=node_parser,
    transformations=transformations,
    store_nodes_override=True
)


In [16]:
from llama_index.core import StorageContext, load_index_from_storage
from pathlib import Path
# ============ 保存 Milvus 索引 ============
def save_milvus_index(index, persist_dir="./milvus_storage"):
    """保存Milvus索引（持久化到本地）"""
    persist_path = Path(persist_dir)
    persist_path.mkdir(parents=True, exist_ok=True)
    
    # LlamaIndex会自动保存索引结构和docstore
    index.storage_context.persist(persist_dir=persist_dir)
    
    print(f"✅ 索引已保存到: {persist_dir}")
    
    # 保存索引元信息
    index_info = {
        'collection_name': 'rag_collection',
        'milvus_db_path': abs_db_path,
        'embedding_dim': dimension,
        'total_documents': len(index.docstore.docs),
        'index_type': 'VectorStoreIndex'
    }
    
    info_file = persist_path / "index_info.json"
    with open(info_file, 'w', encoding='utf-8') as f:
        json.dump(index_info, f, ensure_ascii=False, indent=2)
    print(f"✅ 索引信息已保存到: {info_file}")
    print(f"📊 索引信息: {index_info}")

# ============ 使用示例 ============
# 保存索引
save_milvus_index(index, persist_dir="./milvus_storage")

# 加载索引
# index = load_milvus_index(persist_dir="./milvus_storage", milvus_db_path=abs_db_path)


✅ 索引已保存到: ./milvus_storage
✅ 索引信息已保存到: milvus_storage/index_info.json
📊 索引信息: {'collection_name': 'rag_collection', 'milvus_db_path': '/root/milvus_test/milvus_lite.db', 'embedding_dim': 1024, 'total_documents': 2493, 'index_type': 'VectorStoreIndex'}


### 非第一次运行 加载持久化运行

In [6]:
import pickle
import json
from pathlib import Path
def load_nodes(save_dir="./saved_data"):
    """加载节点数据"""
    save_path = Path(save_dir)
    pickle_file = save_path / "nodes.pkl"
    
    if not pickle_file.exists():
        raise FileNotFoundError(f"❌ 找不到节点文件: {pickle_file}")
    
    with open(pickle_file, 'rb') as f:
        nodes = pickle.load(f)
    
    print(f"✅ 已加载 {len(nodes)} 个节点")
    
    # 验证数据
    print(f"📊 节点验证:")
    print(f"  - 总节点数: {len(nodes)}")
    return nodes

In [7]:
from llama_index.core import StorageContext, load_index_from_storage
from pathlib import Path
# ============ 加载 Milvus 索引 ============
def load_milvus_index(persist_dir="./storage", milvus_db_path=None):
    """加载已保存的Milvus索引"""
    persist_path = Path(persist_dir)
    
    if not persist_path.exists():
        raise FileNotFoundError(f"❌ 找不到索引目录: {persist_dir}")
    
    # 读取索引信息
    info_file = persist_path / "index_info.json"
    if info_file.exists():
        with open(info_file, 'r', encoding='utf-8') as f:
            index_info = json.load(f)
        print(f"📊 索引信息: {index_info}")
        milvus_db_path = milvus_db_path or index_info.get('milvus_db_path')
    
    # 重建 Milvus vector store
    milvus_vector_store = MilvusVectorStore(
        uri=milvus_db_path,
        collection_name="rag_collection",
        dim=dimension,
        overwrite=False  # 不覆盖已有数据
    )
    
    # 重建 storage context
    storage_context = StorageContext.from_defaults(
        vector_store=milvus_vector_store,
        persist_dir=persist_dir
    )
    
    # 加载索引
    index = load_index_from_storage(
        storage_context=storage_context,
        embed_model=Settings.embed_model
    )
    
    print(f"✅ 索引已加载")
    print(f"  - 文档数量: {len(index.docstore.docs)}")
    
    return index



In [8]:
nodes = load_nodes(save_dir="./saved_nodes")
index = load_milvus_index(persist_dir="./milvus_storage", milvus_db_path=abs_db_path)
index.embed_model=Settings.embed_model

✅ 已加载 2493 个节点
📊 节点验证:
  - 总节点数: 2493
📊 索引信息: {'collection_name': 'rag_collection', 'milvus_db_path': '/root/milvus_test/milvus_lite.db', 'embedding_dim': 1024, 'total_documents': 2493, 'index_type': 'VectorStoreIndex'}


/root/miniconda3/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-10-11 09:49:37,774 - INFO - Loading all indices.


Loading llama_index.core.storage.kvstore.simple_kvstore from ./milvus_storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./milvus_storage/index_store.json.
✅ 索引已加载
  - 文档数量: 2493


In [9]:
# 单次快速检索
retriever = index.as_retriever(similarity_top_k=3)
nodes_test = retriever.retrieve("腾讯游戏 三角洲行动")


# 打印结果
for i, node in enumerate(nodes_test, 1):
    print(f"\n[{i}] 分数: {node.score:.4f} | 文件: {node.metadata.get('file_name')}")
    print(f"内容: {node.text[:200]}")


[1] 分数: 0.5139 | 文件: None
内容: · 我們升級了小遊戲的技術底座，能夠兼容更多的遊戲引擎，提升了圖像渲染效果，降低了加載時長，助⼒遊戲 開發者將複雜的手機遊戲應⽤適配至小遊戲。⼆零⼆五年第⼆季小遊戲的總流水同比增長 20% 。

· 本土遊戲方面， 《三角洲行動》 是我們⼆零⼆四年九月在移動端和個人電腦端推出的第-人稱射擊遊戲， ⼆零⼆五年七月的平均日活躍賬戶數突破 2,000 萬，位居行業日活躍賬戶數前五，流水前三 。 1

·

[2] 分数: 0.5011 | 文件: None
内容: 411 1,371 3% 1,402 0.6% QQ 的移动终端月活跃账户数 532 571 -7% 534 -0.4% 收费增值服务付费会员数 264 263 0.4% 268 -1% 3 公司数据， QuestMobile ， Sensor Tower 4 2Q2025 付费会员数的日均值 5 2Q2025 每月最后一日的平均付费会员数 6 发布于 https://huggingface.co

[3] 分数: 0.4681 | 文件: None
内容: 534 亿元。截止 2025 年 6 月 30 日，我们于非上市 投资公司（不包括附属公司）权益的账面价值为人民币 3,423 亿元，相较于截止 2025 年 3 月 31 日的账 面价值为人民币 3,379 亿元。

▪ 本公司于 2Q2025 于香港联交所以约 194 亿港元的总代价回购约 3,888 万股股份。 1 非国际财务报告准则撇除股份酬金、并购带来的效应，如来自投资公司的（收益） 亏


## search and rerank

In [ ]:
# !pip install llama-index-retrievers-bm25
# !pip install llama-index-packs-fusion-retriever

In [10]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.packs.fusion_retriever import HybridFusionRetrieverPack

In [11]:
reranker_model_path = "autodl-tmp/Qwen3-Reranker-4B"

reranker = SentenceTransformerRerank(
    model=reranker_model_path,
    top_n=5,
    device="cuda",
    trust_remote_code=True
)

cross_encoder = reranker._model
reranker_tokenizer = cross_encoder.tokenizer
reranker_model = cross_encoder.model

special_tokens = {'pad_token': '[PAD]'}
num_added_tokens = reranker_tokenizer.add_special_tokens(special_tokens)

reranker_model.resize_token_embeddings(len(reranker_tokenizer))

reranker_tokenizer.pad_token = '[PAD]'
reranker_tokenizer.pad_token_id = reranker_tokenizer.convert_tokens_to_ids('[PAD]')
reranker_model.config.pad_token_id = reranker_tokenizer.pad_token_id

print(f"Pad token: {reranker_tokenizer.pad_token}")
print(f"Pad token ID: {reranker_tokenizer.pad_token_id}")
print(f"Model config pad_token_id: {reranker_model.config.pad_token_id}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at autodl-tmp/Qwen3-Reranker-4B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Pad token: [PAD]
Pad token ID: 151669
Model config pad_token_id: 151669


In [12]:
def reranker_tokenize(text):
    rerank_tokenizer = AutoTokenizer.from_pretrained("autodl-tmp/Qwen3-Reranker-8B", padding_side='left')
    if not text.strip():
        return []
    tokens = rerank_tokenizer.tokenize(text)
    return tokens

In [35]:
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes, 
    similarity_top_k=10,
    tokenizer=reranker_tokenize)
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=10)

2025-10-11 10:37:41,718 - WARNING - The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
2025-10-11 10:37:42,416 - DEBUG - Building index from IDs objects


In [36]:
hybrid_pack = HybridFusionRetrieverPack(
    nodes=nodes,
    bm25_retriever=bm25_retriever,
    vector_retriever=vector_retriever,
    mode="reciprocal_rerank",
    similarity_top_k=20
)
hybrid_retriever = hybrid_pack.fusion_retriever

2025-10-11 10:40:30,352 - DEBUG - Building index from IDs objects


In [31]:
text_qa_template_str = (
    "上下文信息如下：\n"
    "{context_str}\n"
    "基于提供的上下文，用中文直接回答查询，答案只能从上下文知识中获取，不要自己发挥。\n"
    "查询：{query_str}\n"
    "回答："
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "原始查询是：{query_str}\n"
    "我们已有回答：{existing_answer}\n"
    "基于以下新上下文，用中文精炼现有回答，问题的核心回答要放在最前边，然后是解释，确保完整性和准确性：\n"
    "{context_msg}\n"
    "精炼后的回答："
)
refine_template = PromptTemplate(refine_template_str)

In [32]:
response_synthesizer = get_response_synthesizer(
    text_qa_template=text_qa_template,
    refine_template=refine_template,
    response_mode="compact"
)

In [33]:
query_engine = RetrieverQueryEngine(
    retriever=hybrid_retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[reranker]
)

In [45]:
query = """
您是一家大型商业银行的首席合规官。您的一位客户是腾讯的一位高管，由于其家庭关系，他也被列为“外国政要”。在腾讯2025年第二季度财报发布后的一周内，他通过贵行进行了以下交易：

他使用腾讯发行的单位卡购买了一件价值11万元人民币的艺术品，摆放在办公室。

他将8万元人民币现金存入个人账户，并注明这笔资金来自个人股息。

他作为付款人签署了一张金额为600万元人民币的商业承兑汇票，付款期限为90天。该草案旨在为一家3D打印公司提供新的融资，该公司将使用腾讯的“混元3D模型”人工智能服务，该服务在最近的财报中被重点提及。

您的任务：

仅根据提供的文件，回答以下问题。

对于这第一笔交易，请分别找出任何可能违反“支付结算办法”的情况，或任何触发“多家客户尽职调查和客户身份资料及交易记录保存管理办法”强制措施的特定门槛。请引用文件中的具体条款编号来支持您的发现。"""
response = query_engine.query(query)
print(response)

Generated queries:
1. 根据《支付结算办法》第28条和第35条，单位卡单笔交易超过10万元人民币是否需进行额外身份核实与交易背景审查，且该艺术品购买是否属于“异常大额交易”而触发反洗钱监测？
2. 依据《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》第12条和第18条，外国政要关联客户单笔交易超过5万元人民币是否构成“高风险客户”交易，需启动强化尽职调查程序？
3. 结合《客户尽职调查和客户身份资料及交易记录保存管理办法》第15条，对于涉及“外国政要”身份的客户，其通过单位卡进行11万元艺术品购买是否构成“非正常交易目的”并触发强制性客户身份资料保存与报告义务？


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

根据提供的文件内容，对第一笔交易（使用腾讯发行的单位卡购买一件价值11万元人民币的艺术品）进行分析：

1. **关于《支付结算办法》的合规性分析**：
   - 文件中未提及单位卡购买艺术品是否属于禁止或受限的支付结算行为。
   - 交易金额为11万元人民币，未超过任何明确的限额或触发特殊收费或限制条款。
   - 电报费、手续费、邮电费等均按标准收取，未发现违反《支付结算办法》中关于收费或结算流程的规定。
   - 未发现该交易违反《支付结算办法》第二百五十八条、第二百五十九条、第二百六十条等关于执行优先级、解释权和施行时间的规定。

   ✅ **结论**：该交易**不违反《支付结算办法》**。

2. **关于《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》的强制措施触发情况**：
   - 客户身份：该客户为腾讯高管，且因家庭关系被列为“外国政要”（PEP）。
   - 根据《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》**第四十三条**，金融机构在开展客户尽职调查时，应关注客户所在国家或地区风险状况，且对高风险客户（如外国政要）需加强尽职调查。
   - 虽然未明确说明“外国政要”直接触发报告义务，但**第四十三条第（二）项**规定：  
     > “有明显理由怀疑客户建立业务关系的目的和性质与洗钱和恐怖融资等违法犯罪活动相关的”  
     应当报告可疑行为。
   - 该客户为“外国政要”，且交易涉及单位卡购买高价值艺术品（11万元），虽未达大额交易报告标准（如10万元人民币以上现金交易），但结合其身份特殊性，存在较高洗钱或腐败风险。
   - 根据**第四十三条**，若金融机构有理由怀疑该交易目的与洗钱或恐怖融资相关，应向中国反洗钱监测分析中心和中国人民银行当地分支机构报告。

   ✅ **结论**：该交易**触发《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》第四十三条的可疑行为报告义务**，因客户为“外国政要”，且交易性质（单位卡购买高价值艺术品）存在与洗钱或恐怖融资相关的合理怀疑。

---

### 最终回答：
- **违反《支付结算办法》的情况**：无。
- **触发《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》强制措施的特定门槛**：  
  **第四十三条**（因客户为“外

In [42]:
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore, QueryBundle, TextNode
from llama_index.core.retrievers import BaseRetriever
from typing import List, Optional, Dict
import copy

# ==================== 1. 节点分割器 ====================
class NodeSplitter:
    """将长节点分割成多个子节点,保持父子关系"""
    
    def __init__(self, chunk_size: int = 512, overlap_ratio: float = 0.1):
        """
        Args:
            chunk_size: 子节点的目标长度
            overlap_ratio: 重叠比例 (0.1 表示 10%)
        """
        self.chunk_size = chunk_size
        self.overlap_size = int(chunk_size * overlap_ratio)
        
    def split_node(self, node: NodeWithScore, parent_id: str = None) -> List[NodeWithScore]:
        """
        将单个节点分割成多个子节点
        
        Args:
            node: 原始节点
            parent_id: 父节点ID (如果为None,使用node.node.node_id)
            
        Returns:
            子节点列表,每个子节点都保留父节点引用
        """
        text = node.node.text
        text_length = len(text)
        
        # 如果文本长度小于chunk_size,直接返回原节点
        if text_length <= self.chunk_size:
            # 添加父节点ID到metadata
            node.node.metadata['parent_node_id'] = parent_id or node.node.node_id
            node.node.metadata['is_child_node'] = False
            return [node]
        
        parent_node_id = parent_id or node.node.node_id
        child_nodes = []
        start = 0
        chunk_index = 0
        
        while start < text_length:
            end = min(start + self.chunk_size, text_length)
            chunk_text = text[start:end]
            
            # 创建子节点
            child_node = TextNode(
                text=chunk_text,
                metadata={
                    **node.node.metadata,  # 继承父节点的metadata
                    'parent_node_id': parent_node_id,
                    'chunk_index': chunk_index,
                    'is_child_node': True,
                    'parent_text_length': text_length,
                    'chunk_start': start,
                    'chunk_end': end
                },
                excluded_embed_metadata_keys=node.node.excluded_embed_metadata_keys,
                excluded_llm_metadata_keys=node.node.excluded_llm_metadata_keys,
            )
            
            # 保持原始评分
            child_node_with_score = NodeWithScore(
                node=child_node,
                score=node.score
            )
            
            child_nodes.append(child_node_with_score)
            
            # 计算下一个起点 (带重叠)
            start += (self.chunk_size - self.overlap_size)
            chunk_index += 1
        
        return child_nodes
    
    def split_nodes(self, nodes: List[NodeWithScore]) -> tuple[List[NodeWithScore], Dict[str, NodeWithScore]]:
        """
        批量分割节点
        
        Returns:
            (子节点列表, 父节点映射字典)
        """
        all_child_nodes = []
        parent_node_map = {}  # parent_node_id -> 原始父节点
        
        for node in nodes:
            parent_id = node.node.node_id
            parent_node_map[parent_id] = node  # 保存原始父节点
            
            child_nodes = self.split_node(node, parent_id)
            all_child_nodes.extend(child_nodes)
        
        return all_child_nodes, parent_node_map


# ==================== 2. 子节点到父节点的后处理器 ====================
class ChildToParentPostprocessor(BaseNodePostprocessor):
    """
    将rerank后的子节点还原为父节点
    策略: 如果多个子节点来自同一父节点,取最高分的子节点分数作为父节点分数
    """
    
    # 使用 Pydantic 的方式声明字段
    parent_node_map: Dict[str, Any] = {}
    keep_top_k: int = 5
    
    def __init__(self, parent_node_map: Dict[str, NodeWithScore], keep_top_k: int = 5, **kwargs):
        """
        Args:
            parent_node_map: 父节点ID到父节点的映射
            keep_top_k: 最终保留的父节点数量
        """
        # 使用 Pydantic 的初始化方式
        super().__init__(
            parent_node_map=parent_node_map,
            keep_top_k=keep_top_k,
            **kwargs
        )
    
    def _postprocess_nodes(
        self, 
        nodes: List[NodeWithScore], 
        query_bundle: Optional[QueryBundle] = None
    ) -> List[NodeWithScore]:
        """
        将子节点还原为父节点
        """
        # 按父节点ID分组,记录每个父节点的最高分数
        parent_scores: Dict[str, float] = {}
        parent_child_nodes: Dict[str, List[NodeWithScore]] = {}
        
        for node in nodes:
            parent_id = node.node.metadata.get('parent_node_id')
            
            if not parent_id:
                # 如果没有父节点ID,说明是原始节点,直接保留
                parent_scores[node.node.node_id] = node.score
                parent_child_nodes[node.node.node_id] = [node]
                continue
            
            # 记录最高分数
            if parent_id not in parent_scores:
                parent_scores[parent_id] = node.score
                parent_child_nodes[parent_id] = [node]
            else:
                # 取最高分
                parent_scores[parent_id] = max(parent_scores[parent_id], node.score)
                parent_child_nodes[parent_id].append(node)
        
        # 构建父节点列表
        parent_nodes = []
        for parent_id, score in parent_scores.items():
            if parent_id in self.parent_node_map:
                # 使用保存的原始父节点
                parent_node = copy.deepcopy(self.parent_node_map[parent_id])
                parent_node.score = score
                
                # 可选: 在metadata中记录匹配的子节点信息
                child_info = [
                    {
                        'chunk_index': n.node.metadata.get('chunk_index'),
                        'score': n.score,
                        'text_preview': n.node.text[:100]
                    }
                    for n in parent_child_nodes[parent_id]
                ]
                parent_node.node.metadata['matched_children'] = child_info
                
                parent_nodes.append(parent_node)
            else:
                # 如果找不到父节点,使用第一个子节点(不应该发生)
                print(f"警告: 找不到父节点 {parent_id}, 使用子节点代替")
                parent_nodes.append(parent_child_nodes[parent_id][0])
        
        # 按分数排序并返回top_k
        parent_nodes.sort(key=lambda x: x.score, reverse=True)
        return parent_nodes[:self.keep_top_k]
    
    class Config:
        arbitrary_types_allowed = True  # 允许任意类型

# ==================== 3. 自定义检索器包装器 ====================
class SplitNodeRetriever(BaseRetriever):
    """
    包装原始检索器,自动处理节点分割
    """
    
    def __init__(
        self, 
        base_retriever: BaseRetriever,
        chunk_size: int = 512,
        overlap_ratio: float = 0.1
    ):
        """
        Args:
            base_retriever: 原始混合检索器
            chunk_size: 子节点大小
            overlap_ratio: 重叠比例
        """
        super().__init__()
        self.base_retriever = base_retriever
        self.node_splitter = NodeSplitter(chunk_size, overlap_ratio)
        self.parent_node_map = {}
    
    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """
        检索并分割节点
        """
        # 1. 使用原始检索器检索
        nodes = self.base_retriever.retrieve(query_bundle)
        
        # 2. 分割节点
        child_nodes, self.parent_node_map = self.node_splitter.split_nodes(nodes)
        
        print(f"原始节点数: {len(nodes)}, 分割后子节点数: {len(child_nodes)}")
        
        return child_nodes
    
    def get_parent_node_map(self) -> Dict[str, NodeWithScore]:
        """获取父节点映射,供后处理器使用"""
        return self.parent_node_map


def create_parent_postprocessor(retriever: SplitNodeRetriever, keep_top_k: int = 5):
    """动态创建父节点后处理器"""
    return ChildToParentPostprocessor(
        parent_node_map=retriever.get_parent_node_map(),
        keep_top_k=keep_top_k
    )

class DynamicQueryEngine:
    """支持动态后处理器的查询引擎"""
    
    def __init__(self, retriever, response_synthesizer, reranker, keep_top_k=5):
        self.retriever = retriever
        self.response_synthesizer = response_synthesizer
        self.reranker = reranker
        self.keep_top_k = keep_top_k
    
    def query(self, query_str: str):
        from llama_index.core.schema import QueryBundle
        
        # 1. 检索 (自动分割节点)
        query_bundle = QueryBundle(query_str=query_str)
        nodes = self.retriever.retrieve(query_bundle)
        
        # 2. Rerank子节点
        reranked_nodes = self.reranker.postprocess_nodes(nodes, query_bundle)
        
        # 3. 动态创建父节点后处理器
        parent_postprocessor = create_parent_postprocessor(
            self.retriever, 
            keep_top_k=self.keep_top_k
        )
        
        # 4. 还原为父节点
        parent_nodes = parent_postprocessor.postprocess_nodes(reranked_nodes, query_bundle)
        
        # 5. 生成回答
        response = self.response_synthesizer.synthesize(
            query=query_str,
            nodes=parent_nodes
        )
        
        return response

/tmp/ipykernel_1597/2662524485.py:101: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class ChildToParentPostprocessor(BaseNodePostprocessor):


In [43]:
split_retriever = SplitNodeRetriever(
    base_retriever=hybrid_retriever,
    chunk_size=512,      # 分割为512长度 (或256)
    overlap_ratio=0.1    # 10%重叠
)

dynamic_query_engine = DynamicQueryEngine(
    retriever=split_retriever,
    response_synthesizer=response_synthesizer,
    reranker=reranker,
    keep_top_k=5  # 最终返回5个父节点
)


In [44]:
query = """
您是一家大型商业银行的首席合规官。您的一位客户是腾讯的一位高管，由于其家庭关系，他也被列为“外国政要”。在腾讯2025年第二季度财报发布后的一周内，他通过贵行进行了以下交易：

他使用腾讯发行的单位卡购买了一件价值11万元人民币的艺术品，摆放在办公室。

他将8万元人民币现金存入个人账户，并注明这笔资金来自个人股息。

他作为付款人签署了一张金额为600万元人民币的商业承兑汇票，付款期限为90天。该草案旨在为一家3D打印公司提供新的融资，该公司将使用腾讯的“混元3D模型”人工智能服务，该服务在最近的财报中被重点提及。

您的任务：

仅根据提供的文件，回答以下问题。

对于这第一笔交易，请分别找出任何可能违反“支付结算办法”的情况，或任何触发“多家客户尽职调查和客户身份资料及交易记录保存管理办法”强制措施的特定门槛。请引用文件中的具体条款编号来支持您的发现。"""
response = dynamic_query_engine.query(query)
print(response)

Generated queries:
1. 根据《支付结算办法》第28条和第32条，单位卡用于购买高价值艺术品是否构成大额交易或可疑交易？是否存在未按规定进行交易背景审查的情形？
2. 依据《金融机构客户尽职调查和客户身份资料及交易记录保存管理办法》第12条和第17条，外国政要关联客户单笔11万元人民币的非日常交易是否触发强化尽职调查义务？是否符合“大额交易”标准？
3. 结合《支付结算办法》第15条及《客户尽职调查管理办法》第14条，使用单位卡进行非经营性大额消费是否违反单位卡使用范围规定，且是否构成需报告的可疑交易？
原始节点数: 2, 分割后子节点数: 4


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

根据提供的《支付结算办法》内容，对第一笔交易（使用腾讯发行的单位卡购买一件价值11万元人民币的艺术品）进行分析如下：

1. **违反“支付结算办法”第-百四十二条**：
   - 该条款明确规定：“单位卡不得用于１０万元以上的商品交易、劳务供应款项的结算。”
   - 本次交易金额为11万元人民币，超过10万元限额，因此**直接违反**该条款。

2. **触发“客户尽职调查和客户身份资料及交易记录保存管理办法”的强制措施门槛**：
   - 文件中未提及《多家客户尽职调查和客户身份资料及交易记录保存管理办法》的具体内容或相关条款。
   - 因此，**无法根据现有文件判断是否触发该办法中的强制措施门槛**。

综上，仅依据《支付结算办法》：
- **存在明确违反行为**：违反第-百四十二条关于单位卡结算限额的规定。
- **无依据支持触发客户尽职调查相关强制措施**。
